<a href="https://colab.research.google.com/github/bannis14/Portfolio-Projects/blob/main/Time_Series_Forecast/Facebook_Prophet_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facebook Prophet - Parameter Tuning

## Libraries and Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

In [ ]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric
from sklearn.model_selection import ParameterGrid

In [ ]:
# Load the data
# YYYY-MM-DD
df = pd.read_csv('nyc_data.csv')
df.head()

In [ ]:
# Rename variable
df = df.rename(columns={'Demand': 'y',
                        'Date': 'ds'})
df.head(0)

In [ ]:
# Date variable
df.ds = pd.to_datetime(df.ds,
                       format="%m/%d/%Y")
df.ds

## Holiday Handling

In [ ]:
# Easter
dates = pd.to_datetime(df[df.Easter == 1].ds)
easter = pd.DataFrame({'holiday': 'easter',
                       'ds': dates,
                       'lower_window': -5,
                       'upper_window': 2})

In [ ]:
# Thanksgiving
dates = pd.to_datetime(df[df.Thanksgiving == 1].ds)
thanksgiving = pd.DataFrame({'holiday': 'thanksgiving',
                             'ds': dates,
                             'lower_window': -3,
                             'upper_window': 6})
thanksgiving

In [ ]:
# Combining events
holidays = pd.concat([easter, thanksgiving])
holidays

In [ ]:
df = df.drop(columns=["Easter", "Thanksgiving"])
df.head(0)

## Facebook Prophet Model

In [ ]:
# FB model
m = Prophet(holidays=holidays,
            seasonality_mode='multiplicative',
            seasonality_prior_scale=10,
            holidays_prior_scale=10,
            changepoint_prior_scale=0.05)
m.add_regressor('Christmas')
m.add_regressor('Temperature')
m.add_regressor('Marketing')
m.fit(df)

In [ ]:
# Check how many observations
df.shape[0] - 180

In [ ]:
# Cross-validation
df_cv = cross_validation(m,
                         horizon='31 days',
                         period='16 days',
                         initial='2012 days',
                         parallel='processes')
df_cv.head()

In [ ]:
# Performance
performance_metrics(df_cv).head()

In [ ]:
# RMSE and MAPE
print("RMSE: ", round(performance_metrics(df_cv)["rmse"].mean(), 1))
print("MAPE: ", 100 * round(performance_metrics(df_cv)["mape"].mean(), 3), "%")

In [ ]:
# Plotting
plot_cross_validation_metric(df_cv, metric='rmse');

## Parameter Tuning

In [ ]:
# Parameter Grid
param_grid = {'seasonality_mode': ["additive", 'multiplicative'],
              'seasonality_prior_scale': [1, 5, 10, 20],
              'holidays_prior_scale': [5, 10, 20, 25],
              'changepoint_prior_scale': [0.005, 0.01, 0.05, 0.1]}
grid = ParameterGrid(param_grid)
len(list(grid))

In [ ]:
# Store the results
rmse = []

In [ ]:
# Loop
i = 1
for params in grid:
    print(f"{i} / {len(list(grid))}")
    # Model
    m = Prophet(holidays=holidays,
                seasonality_mode=params['seasonality_mode'],
                seasonality_prior_scale=params['seasonality_prior_scale'],
                holidays_prior_scale=params['holidays_prior_scale'],
                changepoint_prior_scale=params['changepoint_prior_scale'])
    m.add_regressor('Christmas')
    m.add_regressor('Temperature')
    m.add_regressor('Marketing')
    m.fit(df)

    # CV
    df_cv = cross_validation(m,
                             horizon='31 days',
                             period='16 days',
                             initial='2012 days',
                             parallel='processes')

    # Measure the error and store the result
    error = performance_metrics(df_cv)["rmse"].mean()
    rmse.append(error)

    i += 1

In [ ]:
# Check the results
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

In [ ]:
# Export best parameters
best_params = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_params.to_csv("Forecasting Product/best_params_prophet.csv")